In [11]:
%pip install networkx matplotlib seaborn plotly
%pip install gurobipy
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
from datetime import datetime, timedelta
import random
import csv
import plotly.express as px
import io
from config import gurobiWLSACCESSID,gurobiWLSSECRET,gurobiLICENSEID

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [13]:
df = pd.read_csv('dados.csv')
df_clean = pd.read_csv('dados.csv')
print(df)

    Contêiner Caixa  Cilindro  Peso do cilindro (g)  Volume do cilindro (mL)
0           A  LB_1         1                   294                   140.00
1           A  LB_1         2                   832                    94.87
2           B  LB_1         1                   977                   101.14
3           B  LB_1         2                    57                     8.88
4           B  LB_1         3                   999                   135.55
..        ...   ...       ...                   ...                      ...
406        AQ  LB_1         5                    99                    13.62
407        AQ  LB_1         6                   423                   227.42
408        AQ  LB_1         7                   114                    13.33
409        AQ  LB_1         8                     5                     0.87
410        AQ  LB_1         9                     5                     1.03

[411 rows x 5 columns]


Datasiens do conjunto de dados

In [14]:
containers = df['Contêiner'].unique()
total_containers = len(containers)
total_boxes = 0
total_cylinders = 0

for container in containers:
  container_data = df[df['Contêiner'] == container]
  boxes = container_data['Caixa'].unique()
  container_cylinders = len(container_data)
  container_weight = container_data['Peso do cilindro (g)'].sum()
  container_volume = container_data['Volume do cilindro (mL)'].sum()

  total_boxes += len(boxes)
  total_cylinders += len(container_data)

  print(f"Contêiner {container}:")
  print(f"  - Caixas {len(boxes)}: {list(boxes)}")
  print(f"  - Total de cilindros: {container_cylinders}")
  print(f"  - Peso total: {container_weight:.2f} g")
  print(f"  - Volume total: {container_volume:.2f} mL")
  print()

print(f'São:\n    {total_containers} contêineres\n    {total_boxes} caixas\n    {total_cylinders} cilindros')

Contêiner A:
  - Caixas 1: ['LB_1']
  - Total de cilindros: 2
  - Peso total: 1126.00 g
  - Volume total: 234.87 mL

Contêiner B:
  - Caixas 2: ['LB_1', 'LB_2']
  - Total de cilindros: 9
  - Peso total: 5172.00 g
  - Volume total: 1366.01 mL

Contêiner C:
  - Caixas 2: ['LB_1', 'LB_2']
  - Total de cilindros: 4
  - Peso total: 356.00 g
  - Volume total: 96.38 mL

Contêiner D:
  - Caixas 2: ['LB_1', 'LB_2']
  - Total de cilindros: 5
  - Peso total: 3996.00 g
  - Volume total: 895.03 mL

Contêiner F:
  - Caixas 1: ['LB_1']
  - Total de cilindros: 10
  - Peso total: 7334.00 g
  - Volume total: 1692.48 mL

Contêiner G:
  - Caixas 1: ['LB_1']
  - Total de cilindros: 2
  - Peso total: 406.00 g
  - Volume total: 58.39 mL

Contêiner H:
  - Caixas 2: ['LB_1', 'LB_2']
  - Total de cilindros: 16
  - Peso total: 826.00 g
  - Volume total: 227.24 mL

Contêiner I:
  - Caixas 1: ['LB_1']
  - Total de cilindros: 7
  - Peso total: 3701.00 g
  - Volume total: 1061.57 mL

Contêiner J:
  - Caixas 2: ['LB_

Arrumar df para criar dicionários
- Caixas com mesmo nome em contêiners diferentes é um problema
- Cilindros com o mesmo nome em caixas diferentes é um problema

Os padrões
- Caixas de LB_1 até LB_59
- Cilindros de 1 até 411

In [15]:
i = 1
for a in range(len(df_clean)):
  df_clean['Cilindro'][a] = a + 1
  if a == 0 or a == 410:
    label = f"LB_{i}"
  elif df_clean['Contêiner'][a] != df_clean['Contêiner'][a-1] or df['Caixa'][a] != df['Caixa'][a-1]:
    i = i + 1
    label = f"LB_{i}"

  df_clean['Caixa'][a] = label
print(df_clean)

print("\nUnique values")
containers = df_clean['Contêiner'].unique().tolist()
print(f"Contêiners:\n{containers}")
print(f"Size: {len(containers)}")
boxes = df_clean['Caixa'].unique().tolist()
print(f"Caixas:\n{boxes}")
print(f"Size: {len(boxes)}")
cylinders = df_clean['Cilindro'].tolist()
print(f"Cilindros:\n{cylinders}")
print(f"Size: {len(cylinders)}")

print()
boxes_in_container = {c: df_clean[df_clean['Contêiner'] == c]['Caixa'].unique().tolist() for c in containers}
print(f"boxes_in_container: {boxes_in_container}")
cylinders_in_container = {c: df_clean[df_clean['Contêiner'] == c]['Cilindro'].unique().tolist() for c in containers}
print(f"cylinders_in_container: {cylinders_in_container}")
container_of_box = df_clean.set_index('Caixa')['Contêiner'].to_dict()
print(f"container_of_box: {container_of_box}")
cylinders_in_box = {b: df_clean[df_clean['Caixa'] == b]['Cilindro'].unique().tolist() for b in boxes}
print(f"cylinders_in_box: {cylinders_in_box}")
container_of_cylinder = df_clean.set_index('Cilindro')['Contêiner'].to_dict()
print(f"container_of_cylinder: {container_of_cylinder}")
box_of_cylinder = df_clean.set_index('Cilindro')['Caixa'].to_dict()
print(f"box_of_cylinder: {box_of_cylinder}")
cylinder_data = df_clean.set_index('Cilindro')[['Peso do cilindro (g)', 'Volume do cilindro (mL)']].to_dict()
print(f"cylinder_data: {cylinder_data}")

    Contêiner  Caixa  Cilindro  Peso do cilindro (g)  Volume do cilindro (mL)
0           A   LB_1         1                   294                   140.00
1           A   LB_1         2                   832                    94.87
2           B   LB_2         3                   977                   101.14
3           B   LB_2         4                    57                     8.88
4           B   LB_2         5                   999                   135.55
..        ...    ...       ...                   ...                      ...
406        AQ  LB_59       407                    99                    13.62
407        AQ  LB_59       408                   423                   227.42
408        AQ  LB_59       409                   114                    13.33
409        AQ  LB_59       410                     5                     0.87
410        AQ  LB_59       411                     5                     1.03

[411 rows x 5 columns]

Unique values
Contêiners:
['A', 'B', 'C

Criar um modelo na GUROBI

In [ ]:
params = {
  "WLSACCESSID": gurobiWLSACCESSID,
  "WLSSECRET": gurobiWLSSECRET,
  "LICENSEID": gurobiLICENSEID,
}
env = gp.Env(params=params)

# Create the model within the Gurobi environment
model = gp.Model(env=env)

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2709121
Academic license 2709121 - for non-commercial use only - registered to vi___@usp.br


# Variáveis
$x$ são os contêiners

$y$ são as caixas

$z$ são os cilindros

#### Representação

1o contêiner

$\ \ \ \ x_{1} \supset y_{1} \supset z_{1}, z_{2}$

Condição a $\rightarrow x_1 = 1$

$\ \ \ \ x_1 <= y_1 \rightarrow$ contêiner só pode ser escolhido se escolher caixa (Restr1)

$\ \ \ \ y_1 <= z_1 + z_2 \rightarrow$ caixa só pode ser escolhida se escolher pelo menos um cilindro da caixa (Restr2)

Condição b $\rightarrow y_1 = 1$

$\ \ \ \ y_1 <= x_1 \rightarrow$ se escolhi a caixa tenho que escolher o contêiner (Restr3)

$\ \ \ \ y_1 <= z_1 + z_2 \rightarrow$ se escolhi a caixa tenho que escolher pelo menos um cilindro da caixa (Restr2)

Condição c $\rightarrow z_1 = 1\ |\ z_2 = 1$

$\ \ \ \ z_{1-2} <= y_1 \rightarrow$ cilindro só pode ser escolhido se escolher caixa (Restr4)

$\ \ \ \ y_1 <= x_1 \rightarrow$ caixa só pode ser escolhida se escolher contêiner (Restr3)

Condição d $\rightarrow x_1 = 0$

$\ \ \ \ x_1 >= y_1 \rightarrow$ se não escolher contêiner, não escolhe caixa (Restr3)

$\ \ \ \ x_1 >= z_{1-2} \rightarrow$ se não escolher contêiner, não escolhe cilindros (Restr5)

---
2o contêiner

$\ \ \ \ x_{2} \supset y_{2} \supset z_{3}, z_{4}, z_{5}, z_{6}, z_{7}, z_{8}, z_{9}$

$\ \ \ \ x_{2} \supset y_{3} \supset z_{10}, z_{11}$

Condição a $\rightarrow x_2 = 1$

$\ \ \ \ x_2 <= y_2 \rightarrow$ contêiner só pode ser escolhido se escolher caixa (Restr1)

$\ \ \ \ y_2 <= z_{3}+z_{4}+z_{5}+z_{6}+z_{7}+z_{8}+z_{9} \rightarrow$ caixa só pode ser escolhida se escolher pelo menos um cilindro da caixa (Restr2)

Condição b

$\rightarrow y_2 = 1$

$\ \ \ \ y_2 <= x_2 \rightarrow$ se escolhi a caixa tenho que escolher o contêiner (Restr3)

$\ \ \ \ y_2 <= z_{3}+z_{4}+z_{5}+z_{6}+z_{7}+z_{8}+z_{9} \rightarrow$ se escolhi a caixa tenho que escolher pelo menos um cilindro da caixa (Restr2)

$\rightarrow y_3 = 1$

$\ \ \ \ y_3 <= x_2 \rightarrow$ se escolhi a caixa tenho que escolher o contêiner (Restr3)

$\ \ \ \ y_3 <= z_{10}+z_{11} \rightarrow$ se escolhi a caixa tenho que escolher pelo menos um cilindro da caixa (Restr2)

Condição c

$\rightarrow z_{3}=1|z_{4}=1|z_{5}=1|z_{6}=1|z_{7}=1|z_{8}=1|z_{9}=1$

$\ \ \ \ z_{3-9} <= y_2 \rightarrow$ cilindro só pode ser escolhido se escolher caixa (Restr4)

$\ \ \ \ y_2 <= x_2 \rightarrow$ caixa só pode ser escolhida se escolher contêiner (Restr3)

$\rightarrow z_{10}=1|z_{11}=1$

$\ \ \ \ z_{10-11} <= y_3 \rightarrow$ cilindro só pode ser escolhido se escolher caixa (Restr4)

$\ \ \ \ y_3 <= x_2 \rightarrow$ caixa só pode ser escolhida se escolher contêiner (Restr3)

Condição d $\rightarrow x_2 = 0$

$\ \ \ \ x_2 >= y_{2-3} \rightarrow$ se não escolher contêiner, não escolhe caixa (Restr3)

$\ \ \ \ x_2 >= z_{3-11} \rightarrow$ se não escolher contêiner, não escolhe cilindros (Restr5)

----

# Model

maximize

$\ \ \ \ 1\ (problema\ de\ satisfação)$

subject to

  $\ \ \ \ (Restrição\ c)\ z_k <= x_i$

  $\ \ \ \ (Restrição\ c)\ y_j <= x_i$

  $\ \ \ \ (Restrição\ b)\ y_j <= \sum_{k}z$

  $\ \ \ \ (Restrição\ a)\ x_i <= \sum_{j}y$

  $\ \ \ \ (Restrição\ a)\ x_i <= \sum_{k}z$

  $\ \ \ \ (Restrição\ e)\ \sum_{i=1}^{40} x_{i} = 35$

  $\ \ \ \ (Restrição\ f)\ \sum_{k=1}^{411} z_{k}*cylinder\_volume_{k} = 5163.69$

  $\ \ \ \ (Restrição\ g)\ \sum_{k=1}^{411} z_{k}*cylinder\_weight_{k} = 18844$

  $\ \ \ \ x_i\ binary\ (i\ range\ [1,40])$

  $\ \ \ \ y_j\ binary\ (j\ range\ [1,59])$
  
  $\ \ \ \ z_k\ binary\ (k\ range\ [1,411])$

In [ ]:
#!/usr/bin/env python3.11


try:
    x = model.addVars(containers, vtype=GRB.BINARY, name="container")
    y = model.addVars(boxes, vtype=GRB.BINARY, name="box")
    z = model.addVars(cylinders, vtype=GRB.BINARY, name="cylinder")

    # Restrição e
    model.addConstr(x.sum() == 35, "exact_35_containers")
    # Restrição f
    volume_constr = 0
    for cyl in cylinders:
      vol = z[cyl] * cylinder_data['Volume do cilindro (mL)'][cyl]
      volume_constr = volume_constr + vol
    model.addConstr(volume_constr == 5163.69, "f_total_volume")
    # Restrição g
    weight_constr = gp.quicksum(z[cyl] * cylinder_data['Peso do cilindro (g)'][cyl] for cyl in cylinders)
    model.addConstr(weight_constr == 18844, "g_total_weight")
    # Restrição c
    for cyl in cylinders:
      box = box_of_cylinder[cyl]
      container = container_of_box[box]
      model.addConstr(z[cyl] <= y[box], f"c_cylinder_{cyl}_to_box")
      model.addConstr(y[box] <= x[container], f"c_box_{box}_to_container")
    # Restrição b
    for box in boxes:
      container = container_of_box[box]
      # Container deve ser selecionado (já coberto por y[box] <= x[container] acima)
      # Pelo menos 1 cilindro na caixa se ela for selecionada
      model.addConstr(y[box] <= gp.quicksum(z[cyl] for cyl in cylinders_in_box[box]), f"b_box_{box}_has_cylinder")
    # Restrição a
    for container in containers:
      # Pelo menos 1 caixa no container
      model.addConstr(x[container] <= gp.quicksum(y[box] for box in boxes_in_container[container]), f"a_container_{container}_has_box")
      # Pelo menos 1 cilindro no container
      model.addConstr(x[container] <= gp.quicksum(z[cyl] for cyl in cylinders_in_container[container]), f"a_container_{container}_has_cylinder")
    # Restrição d
    # coberta nas outras restrições

    model.setObjective(1, GRB.MINIMIZE)

    # Optimize model
    model.optimize()

    if model.status == GRB.OPTIMAL:
      print("\nCilindros selecionados:")
      selected_count = 0
      with open("./solution.csv","w") as f:
        f.write("Contêiner,Caixa,Cilindro,Peso do cilindro (g),Volume do cilindro (mL)")
        for cyl in cylinders:
            if z[cyl].X > 0.5:
                selected_count += 1
                box = box_of_cylinder[cyl]
                container = container_of_box[box]
                weight = cylinder_data['Peso do cilindro (g)'][cyl]
                volume = cylinder_data['Volume do cilindro (mL)'][cyl]
                print(f"Contêiner {container}, Caixa {box}, Cilindro {cyl}, Peso: {weight}g, Volume: {volume}mL")
                f.write(f"\n{container},{box},{cyl},{weight},{volume}")

      print(f"\nTotal de cilindros selecionados: {selected_count}")
      print(f"Total de containers selecionados: {sum(x[c].X > 0.5 for c in containers)}")
      print(f"Total de caixas selecionadas: {sum(y[b].X > 0.5 for b in boxes)}")

      # Verificar totais
      total_weight = sum(z[cyl].X * cylinder_data['Peso do cilindro (g)'][cyl] for cyl in cylinders)
      total_volume = sum(z[cyl].X * cylinder_data['Volume do cilindro (mL)'][cyl] for cyl in cylinders)
      print(f"\nPeso total: {total_weight:.2f}g (meta: 18844g)")
      print(f"Volume total: {total_volume:.2f}mL (meta: 5163.69mL)")
    else:
        print("Não foi encontrada solução ótima.")
        print(f"Status: {model.status}")

except gp.GurobiError as e:
    print(f"Error code {e.errno}: {e}")

except AttributeError:
    print("Encountered an attribute error")

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Academic license 2709121 - for non-commercial use only - registered to vi___@usp.br
Optimize a model with 3856 rows, 2040 columns and 14104 nonzeros
Model fingerprint: 0xa4b1bd70
Variable types: 0 continuous, 2040 integer (2040 binary)
Coefficient statistics:
  Matrix range     [5e-01, 1e+03]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [4e+01, 2e+04]

MIP start from previous solve did not produce a new incumbent solution

Presolve removed 3353 rows and 1573 columns
Presolve time: 0.02s
Presolved: 503 rows, 467 columns, 2410 nonzeros
Variable types: 0 continuous, 467 integer (467 binary)

Root relaxation: objective 1.000000e+00, 123 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |   